In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
import copy
import time
from sklearn.preprocessing import StandardScaler
import random

In [2]:
df = pd.read_csv("../Data/data_(8-8).csv")
sc = StandardScaler()

In [3]:
X = df.iloc[:800, :8].values
y = df.iloc[:800, 8:16].values
X = sc.fit_transform(X)
X_test = df.iloc[800:1000, :8].values
y_test = df.iloc[800:1000, 8:16].values
X_test = sc.fit_transform(X_test)

In [36]:
X1= np.reshape(X, (X.shape[0],2, int(X.shape[1]/2)))
y1= np.reshape(y, (y.shape[0], y.shape[1]))
X2= np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y2= np.reshape(y_test, (y_test.shape[0], y_test.shape[1]))

In [39]:
model = Sequential()
model.add(LSTM(units=24, input_shape=(2,4), activation="tanh"))
model.add(Dense(8, activation="sigmoid")) 
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 24)                2784      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 200       
Total params: 2,984
Trainable params: 2,984
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X1, y1, epochs = 100,batch_size=64)

Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 0.2452 - accuracy: 0.1875
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2389 - accuracy: 0.2962
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2325 - accuracy: 0.3887
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2256 - accuracy: 0.4150
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2180 - accuracy: 0.4200
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2095 - accuracy: 0.4200
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2001 - accuracy: 0.4112
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1897 - accuracy: 0.4100
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1786 - accuracy: 0.4112
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1673 - accuracy: 0.4162
Epoch 11/

13/13 [==============================] - 0s 2ms/step - loss: 0.0345 - accuracy: 0.6212
Epoch 84/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0343 - accuracy: 0.6237
Epoch 85/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.6212
Epoch 86/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0339 - accuracy: 0.6200
Epoch 87/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0336 - accuracy: 0.6200
Epoch 88/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0334 - accuracy: 0.6162
Epoch 89/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0332 - accuracy: 0.6187
Epoch 90/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0330 - accuracy: 0.6175
Epoch 91/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0328 - accuracy: 0.6150
Epoch 92/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0326 - accuracy: 0.6175
Epoch 93/100


In [41]:
units=24

In [42]:
W = model.get_weights()[0]
U = model.get_weights()[1]
b = model.get_weights()[2]
Wy = model.get_weights()[3].T
by = model.get_weights()[4]

W_i = W[:, :units]
W_f = W[:, units: units * 2]
W_c = W[:, units * 2: units * 3]
W_o = W[:, units * 3:]

U_i = U[:, :units]
U_f = U[:, units: units * 2]
U_c = U[:, units * 2: units * 3]
U_o = U[:, units * 3:]

bi = b[:units]
bf = b[units: units * 2]
bc = b[units * 2: units * 3]
bo = b[units * 3:]

In [43]:
W_i.shape

(4, 24)

In [44]:
U_i.shape

(24, 24)

In [45]:
bi.shape

(24,)

In [89]:
bi.reshape(-1,1).shape

(24, 1)

In [47]:
Wi = np.concatenate((U_i.T, W_i.T), axis=1)
Wf = np.concatenate((U_f.T, W_f.T), axis=1)
Wc = np.concatenate((U_c.T, W_c.T), axis=1)
Wo = np.concatenate((U_o.T, W_o.T), axis=1)

In [48]:
Wi.shape

(24, 28)

In [49]:
Wf.shape

(24, 28)

In [50]:
Wc.shape

(24, 28)

In [51]:
Wo.shape

(24, 28)

In [52]:
Wy.shape

(8, 24)

In [53]:
x=X1[0]

In [54]:
x.shape

(2, 4)

In [74]:
xt=x[0].reshape(-1,1)

In [75]:
xt.shape

(4, 1)

In [76]:
from rnn_utils import *

In [77]:
a_prev = np.zeros((24,1))
c_prev = np.zeros((24,1))

In [78]:
n_x, m = xt.shape
n_y, n_a = Wy.shape

In [80]:
c_next = c_prev
a_next = a_prev
for i in range(2):
    concat = np.zeros((n_a + n_x, m))
    concat[: n_a, :] = a_next
    concat[n_a :, :] = x[i].reshape(-1,1)
    ft = sigmoid(np.dot(Wf, concat) + bf.reshape(-1,1))
    it = sigmoid(np.dot(Wi, concat) + bi.reshape(-1,1))
    cct = np.tanh(np.dot(Wc, concat) + bc.reshape(-1,1))
    c_next = ft * c_next + it * cct
    ot = sigmoid(np.dot(Wo, concat) + bo.reshape(-1,1))
    a_next = ot * np.tanh(c_next)

In [81]:
a_next.shape

(24, 1)

In [82]:

# Compute prediction of the LSTM cell (≈1 line)
yt_pred = sigmoid(np.dot(Wy, a_next) + by.reshape(-1,1))

In [92]:
yt_pred.reshape(-1)

array([0.20390019, 0.04551166, 0.72710706, 0.04300485, 0.28760855,
       0.06351687, 0.10023416, 0.51070848])

In [84]:
model.predict(X1[0:1,:,:])

array([[0.20390019, 0.04551166, 0.7271069 , 0.04300481, 0.28760862,
        0.06351683, 0.10023412, 0.51070833]], dtype=float32)

In [85]:
bh = np.random.randn(24, 1)

In [86]:
bh=bh.reshape(-1)

In [87]:
bh.shape

(24,)

In [91]:
y1[i]

array([0, 1, 0, 0, 0, 0, 1, 0], dtype=int64)